In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


# Se levanta el split hecho en el NoteBook anterior

In [2]:
X_train = pd.read_csv("X_train")
X_train.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,...,ciudad_7,ciudad_8,ciudad_9,ciudad_10,anio_publ,cuenta,planta,cuarto,tiene,baño
0,0.0,3.0,2.0,3.0,196.0,160.0,NaN,NaN,NaN,0.0,...,1,1,1,1,2014,0.0,0.0,0.0,0.0,0.0
1,20.0,4.0,2.0,2.0,245.0,196.0,104734.0,NaN,NaN,0.0,...,0,0,0,0,2012,0.0,0.0,0.0,0.0,0.0
2,5.0,3.0,1.0,2.0,108.0,120.0,58952.0,NaN,NaN,0.0,...,0,1,0,1,2015,0.0,0.0,0.0,0.0,0.0
3,10.0,3.0,1.0,3.0,200.0,147.0,49066.0,NaN,NaN,0.0,...,1,0,1,0,2016,0.0,1.0,0.0,0.0,0.0
4,NaN,NaN,0.0,NaN,NaN,250.0,24892.0,19.23339,-99.166991,0.0,...,1,0,1,0,2016,0.0,0.0,0.0,0.0,0.0


In [3]:
X_test = pd.read_csv("X_test")

In [4]:
y_train = pd.read_csv("y_train")

In [5]:
y_test = pd.read_csv("y_test")

# Se levantan todos los XGBoost guardados

In [6]:
xgb_created = []

## Atencion! n tiene que coincidir con la cantidad guardada!

In [7]:
n = 100

In [8]:
for i in range(n):
    current_xgb = xgb.XGBRegressor()
    current_xgb.load_model("xgb_{}".format(i))
    xgb_created.append(current_xgb)

# Se arma el DF de precios para entrenar el RF

In [9]:
XGB_ens_train = pd.DataFrame(index = X_train.index.copy())

In [10]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_train)
    XGB_ens_train['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
XGB_ens_train.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,1731590.375,1.687190e+06,1708003.750,1.777730e+06,1611353.250,1685426.375,1805369.375,1.737057e+06,1.724486e+06,1.715475e+06,...,1.768250e+06,1.722863e+06,1762664.000,1.768310e+06,1.819990e+06,1853000.125,1861383.500,1.779566e+06,1.795215e+06,1776192.000
1,1861900.750,1.811220e+06,1833665.875,1.901733e+06,1891049.375,1797504.375,1612934.125,1.669204e+06,1.775289e+06,1.768255e+06,...,1.454514e+06,1.805304e+06,1809681.125,1.840597e+06,1.724257e+06,1820837.125,1700720.500,1.613766e+06,1.632273e+06,1756345.500
2,849920.375,9.141216e+05,764238.875,7.420772e+05,914536.625,917239.875,820651.000,9.734854e+05,7.981991e+05,9.497426e+05,...,8.563761e+05,9.187719e+05,1052048.250,9.042729e+05,8.909278e+05,820594.375,806802.500,8.459802e+05,7.979854e+05,869928.750
3,1854745.125,1.846927e+06,1866113.750,2.002066e+06,1790563.375,1752173.500,1905055.250,1.927715e+06,1.755575e+06,1.823570e+06,...,1.733450e+06,1.873017e+06,1970537.250,1.770525e+06,1.734895e+06,1896902.625,1909469.500,1.757820e+06,1.954052e+06,2038684.125
4,1172485.375,1.489644e+06,1405968.000,1.665526e+06,1913289.000,2042179.625,1899234.125,1.338867e+06,1.773599e+06,2.308966e+06,...,1.928238e+06,1.012067e+06,1715285.500,2.078893e+06,1.590580e+06,1685367.000,1949662.625,1.152652e+06,2.274063e+06,1052914.375


In [11]:
len(XGB_ens_train)

168000

In [12]:
len(y_train)

168000

## Exporto el XGB_ens_train para poder buscar hiperparametros

In [13]:
XGB_ens_train.to_csv("XGB_ens_train", index = False)

# Entreno el XGB con XGB_ens_train

In [14]:
model_XGB = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9, gamma=5,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=30, missing=None, n_estimators=1600,
       n_jobs=-1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.5, verbosity=1)
model_XGB.fit(XGB_ens_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9, gamma=5,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=30, missing=None, n_estimators=1600,
       n_jobs=-1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.5, verbosity=1)

# Testeo el modelo final

## Armo el DF para X_test

In [15]:
XGB_ens_test = pd.DataFrame(index = X_test.index.copy())


In [16]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test)
    XGB_ens_test['pred_{}'.format(i)] = current_price_pred
XGB_ens_test.head()

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,1.798102e+06,1446248.625,1407567.250,1.653867e+06,1.402460e+06,1442624.500,1.703511e+06,1.649957e+06,1.435246e+06,2.082397e+06,...,1.516377e+06,1.462098e+06,1666077.750,1406809.000,1636870.50,1.727082e+06,1.349754e+06,1.649911e+06,1.931766e+06,1.392859e+06
1,2.560048e+06,2756970.500,2667663.500,2.664034e+06,2.613616e+06,2601715.500,2.486792e+06,2.557274e+06,2.464232e+06,2.712020e+06,...,2.657490e+06,2.554301e+06,2572289.500,2481868.250,2371856.00,2.742639e+06,2.583326e+06,2.616145e+06,2.509907e+06,2.596720e+06
2,2.664327e+06,2246674.750,2728572.750,2.303223e+06,2.519035e+06,2831999.250,2.267160e+06,2.268172e+06,2.876760e+06,2.509534e+06,...,2.332474e+06,1.777443e+06,1496987.875,1867642.875,2871201.75,2.292293e+06,2.148274e+06,1.874294e+06,2.647240e+06,2.183096e+06
3,1.309244e+06,1342845.750,1244864.500,1.376778e+06,1.282279e+06,1426242.000,1.322379e+06,1.186508e+06,1.312307e+06,1.371620e+06,...,1.146656e+06,1.328758e+06,1468304.625,1385828.250,1291130.25,1.421451e+06,1.516852e+06,1.931046e+06,1.090346e+06,1.232119e+06
4,2.889435e+05,277751.500,327708.875,3.096952e+05,3.009607e+05,205150.625,2.084183e+05,2.095406e+05,3.348800e+05,2.344241e+05,...,3.102430e+05,1.753795e+05,380991.875,250429.625,274002.75,2.639580e+05,2.213890e+05,3.714423e+05,2.726218e+05,2.091287e+05


## Se hacen las dos predicciones necesarias

In [17]:
pred_train = model_XGB.predict(XGB_ens_train)
pred_test = model_XGB.predict(XGB_ens_test)

### Metrica de Mean Absolute Error (La de Kaggle) ?? (Hay que preguntar)

In [18]:
mae_train = np.sqrt(sklearn.metrics.mean_absolute_error(y_train, pred_train))
mae_test = np.sqrt(sklearn.metrics.mean_absolute_error(y_test, pred_test))
#print(f"MAE train: {mea_train:.5f}")
#print(f"MAE test: {mea_test:.5f}")
print("MAE train: "+str(mae_train))
print("MAE test: "+str(mae_test))

MAE train: 419.3658542301655
MAE test: 742.4579169351654


#### Overfittea una bocha! Pero eso quiere decir que hay mucho rango para mejorar :)

In [19]:
model_XGB.feature_importances_

array([0.00239181, 0.03313057, 0.00537208, 0.00482643, 0.006119  ,
       0.00314126, 0.02541233, 0.0022669 , 0.00419376, 0.0258463 ,
       0.00209604, 0.00344005, 0.00289741, 0.00897729, 0.00212927,
       0.01869941, 0.01136701, 0.00252756, 0.0362513 , 0.0787347 ,
       0.00256756, 0.00324149, 0.15996659, 0.00251848, 0.00226609,
       0.0058386 , 0.00224689, 0.0073003 , 0.00213188, 0.00243689,
       0.00367606, 0.0084637 , 0.00860469, 0.00257157, 0.00231593,
       0.00220087, 0.01740937, 0.0034225 , 0.00368871, 0.02882289,
       0.00211564, 0.00257606, 0.00314765, 0.01200468, 0.06988376,
       0.00262397, 0.00211326, 0.00224234, 0.00247734, 0.00576335,
       0.00220244, 0.01617857, 0.08630697, 0.00389798, 0.00320891,
       0.00238881, 0.00772143, 0.00352059, 0.00244636, 0.00223055,
       0.01549322, 0.00400271, 0.00225516, 0.00252323, 0.00257986,
       0.00225731, 0.00496355, 0.00241454, 0.00228974, 0.01119674,
       0.00247881, 0.00252556, 0.00390727, 0.00407405, 0.01722

# Se genera el archivo para subir a Kaggle

In [20]:
test_set = pd.read_csv("test_set_xgb.csv")

In [21]:
test_set.columns

Index(['id', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'tipodepropiedad_1', 'tipodepropiedad_2',
       'tipodepropiedad_3', 'tipodepropiedad_4', 'tipodepropiedad_5',
       'tipodepropiedad_6', 'tipodepropiedad_7', 'tipodepropiedad_8',
       'tipodepropiedad_9', 'tipodepropiedad_10', 'tipodepropiedad_11',
       'tipodepropiedad_12', 'tipodepropiedad_13', 'tipodepropiedad_14',
       'tipodepropiedad_15', 'tipodepropiedad_16', 'tipodepropiedad_17',
       'tipodepropiedad_18', 'tipodepropiedad_19', 'tipodepropiedad_20',
       'tipodepropiedad_21', 'tipodepropiedad_22', 'tipodepropiedad_23',
       'tipodepropiedad_24', 'tipodepropiedad_25', 'provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10

In [22]:
X_test_set = test_set.loc[:, test_set.columns != 'id']

## Armo el DF con las predicciones de test

In [23]:
XGB_ens_test_set = pd.DataFrame(index = test_set.index.copy())

In [24]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test_set)
    XGB_ens_test_set['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
XGB_ens_test_set.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,6096182.500,6.374268e+06,6.494796e+06,6156691.000,6.616866e+06,6759101.000,6.071477e+06,7.135716e+06,6.439080e+06,7.026938e+06,...,7449727.50,6337261.000,6.737764e+06,6.794824e+06,7.141736e+06,6.221232e+06,7.007012e+06,6367864.500,6.356058e+06,6.427861e+06
1,1352439.750,1.229247e+06,1.288090e+06,1133989.125,1.130547e+06,1146395.750,1.185653e+06,1.609011e+06,1.244186e+06,1.438133e+06,...,1166266.75,1202930.750,1.088908e+06,1.223164e+06,1.219766e+06,1.084803e+06,1.083353e+06,1390346.125,1.260148e+06,1.182908e+06
2,2466695.250,2.332000e+06,2.240284e+06,2353772.000,2.319975e+06,2121665.750,2.339676e+06,2.344348e+06,2.276065e+06,2.645803e+06,...,2178524.50,2297817.250,2.294977e+06,2.538613e+06,2.195896e+06,2.285500e+06,2.160422e+06,2283872.000,2.228065e+06,2.279858e+06
3,1495220.000,1.754530e+06,1.480402e+06,1458693.500,1.271857e+06,1146683.125,1.593658e+06,1.378543e+06,1.511577e+06,1.530046e+06,...,1416213.50,1272423.125,1.245497e+06,1.317460e+06,1.345033e+06,1.366189e+06,9.900442e+05,1631337.375,1.595380e+06,1.467107e+06
4,610654.625,6.396662e+05,5.978168e+05,620946.625,5.587736e+05,603674.250,5.195907e+05,6.403339e+05,5.708037e+05,5.784116e+05,...,622935.75,582429.125,5.776676e+05,5.981667e+05,4.824537e+05,5.970396e+05,6.073733e+05,644339.375,6.839747e+05,5.814078e+05


In [25]:
pred_a_kaggle = model_XGB.predict(XGB_ens_test_set)

In [26]:
df_a_kaggle = pd.DataFrame()
df_a_kaggle['id'] = test_set['id']
df_a_kaggle['target'] = pred_a_kaggle
df_a_kaggle.head()

,id,target
0,4941,7387005.00
1,51775,1098091.00
2,115253,2518196.25
3,299321,1260658.00
4,173570,598326.50


In [27]:
df_a_kaggle.to_csv("subir_a_kaggle", header = True, index = False)